# Cooperative Game Theory: Shapley Value and Nucleolus

## Introduction

Cooperative game theory studies scenarios where groups of agents, called coalitions, work together to achieve a common goal and need to distribute the payoff among themselves. Two important concepts in this context are the Shapley value and the Nucleolus.

- **Shapley Value**: A method to distribute the total gains to players based on their marginal contributions.
- **Nucleolus**: A method to distribute payoffs by minimizing the maximum dissatisfaction among the players.

In this project, we will explore these concepts, provide historical context, explain the algorithms for computing them, and discuss their time complexities and improvements.

# Historical Background

## Shapley Value

The Shapley value was introduced by Lloyd Shapley in 1953. It provides a fair distribution of payoffs based on the average marginal contributions of each player. The Shapley value has been widely studied and applied in various fields, including economics, political science, and machine learning.

Mathematically, the Shapley value for a player $i$ in a cooperative game $(N, v)$ is given by:

$ \phi_i(v) = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(|N| - |S| - 1)!}{|N|!} \left( v(S \cup \{i\}) - v(S) \right) $

Where:
- $N$ is the set of players.
- $v$ is the value function defining the worth of each coalition.
- $S$ is a subset of $N$ excluding player $i$.

The Shapley value considers all possible orders in which the players can join a coalition and computes the marginal contribution of each player when they join the coalition. By averaging these contributions, we obtain a fair distribution of the total payoff.

## Nucleolus

The Nucleolus was introduced by David Schmeidler in 1969. It focuses on minimizing the maximum dissatisfaction among players to achieve a fair distribution of payoffs. The Nucleolus is particularly useful in situations where fairness is defined by minimizing the worst-case dissatisfaction.

Mathematically, the Nucleolus is the allocation $x \in \mathbb{R}^n$ that lexicographically minimizes the vector of excesses $e(x, S) = v(S) - \sum_{i \in S} x_i$ for all coalitions $S \subseteq N$.

The Nucleolus is found by solving a series of linear programming problems to iteratively minimize the maximum excess (dissatisfaction) for all coalitions. This process ensures that the distribution of payoffs is as fair as possible, according to the defined criteria.

## Theoretical Significance

The Shapley value and Nucleolus are significant in cooperative game theory because they provide different yet complementary approaches to fair distribution:
- The Shapley value is based on the principle of average marginal contributions, emphasizing individual contributions across all coalitions.
- The Nucleolus aims to minimize dissatisfaction, focusing on equity and stability within the coalition structure.


# Game Function

To make the computation of Shapley and Nucleolus values independent of the specific game, we define a `Game` function. This function will describe the value of any coalition of players.

## Example: Network Communication Game

In this example, we consider a network communication scenario where nodes (players) collaborate to transmit data. The value of a coalition of nodes is based on the total data they can transmit together. Each node has a certain capacity, and the total capacity of a coalition is the sum of the capacities of its members.

The value function for an example game with five nodes is defined as follows:

- Node 1: 2 units
- Node 2: 3 units
- Node 3: 5 units
- Node 4: 7 units
- Node 5: 11 units

The value of a coalition is the sum of the capacities of its members.

In [1]:
# Define the Game function for a network communication scenario
def Game(coalition):
    node_capacities = {
        1: 2,
        2: 3,
        3: 5,
        4: 7,
        5: 11
    }
    return sum(node_capacities[node] for node in coalition)

# Shapley Value Algorithm

The Shapley value is computed by considering all possible permutations of players and calculating their marginal contributions. The exact computation has a factorial time complexity, making it impractical for large games. Various approximation methods, such as the Monte Carlo method, have been developed to make the computation feasible.

## Exact Computation

The exact algorithm for the Shapley value involves calculating the marginal contribution of each player for every possible permutation of players.

Mathematically, for a set of players $N$ and a value function $v$, the Shapley value $\phi_i$ for player $i$ is given by:

$ \phi_i(v) = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(|N| - |S| - 1)!}{|N|!} \left( v(S \cup \{i\}) - v(S) \right) $

Where:
- $S$ is a subset of players excluding $i$.
- $v(S)$ is the value of coalition $S$.

**Time Complexity**: The exact computation has a time complexity of $ O(n!) $, where $ n $ is the number of players. This factorial growth makes the exact computation impractical for large numbers of players.

**Advantages**:
- Provides the exact Shapley value.

**Disadvantages**:
- Computationally infeasible for large numbers of players.

To compute the Shapley value exactly, we generate all permutations of players and calculate the marginal contribution of each player in each permutation. The marginal contribution is the difference between the value of the coalition with the player and the value of the coalition without the player.


In [2]:
import itertools
import numpy as np

def calculate_shapley_value(players, game_function):
    n = len(players)
    shapley_values = {player: 0 for player in players}
    
    factorial_n = np.math.factorial(n)
    
    for player in players:
        player_shapley_value = 0
        
        for r in range(n):
            for coalition in itertools.combinations(players - {player}, r):
                coalition_with_player = coalition + (player,)
                
                marginal_contribution = (game_function(coalition_with_player) - game_function(coalition))
                player_shapley_value += marginal_contribution * np.math.factorial(len(coalition)) * np.math.factorial(n - len(coalition) - 1)
        
        shapley_values[player] = player_shapley_value / factorial_n
    
    return shapley_values

# Monte Carlo Method

The Monte Carlo method uses random sampling to approximate the Shapley value. It significantly reduces computational complexity by considering a subset of all possible permutations.

**Time Complexity**: The Monte Carlo method has a time complexity of $ O(S \cdot m) $, where $ S $ is the number of simulations and $ m $ is the complexity of evaluating the value function for each coalition. This method provides a trade-off between accuracy and computation time.

**Advantages**:
- Scalable and flexible.
- Can handle large games by approximating the solution.

**Disadvantages**:
- Accuracy depends on the number of simulations.
- May not converge to the exact Shapley value.

The Monte Carlo method involves generating a number of random permutations of players and computing the marginal contribution of each player in these permutations. The average of these contributions gives an approximation of the Shapley value.


In [3]:
import itertools
import numpy as np
def monte_carlo_shapley(players, game_function, num_simulations=10000):
    n = len(players)
    shapley_values = {player: 0 for player in players}
    
    for _ in range(num_simulations):
        permutation = list(players)
        np.random.shuffle(permutation)
        coalition = set()
        
        for player in permutation:
            prev_value = game_function(coalition)
            coalition.add(player)
            new_value = game_function(coalition)
            shapley_values[player] += (new_value - prev_value) / num_simulations
    
    return shapley_values

# Nucleolus Algorithm

The Nucleolus is computed by finding the allocation that minimizes the maximum excess among all coalitions. This involves solving a series of linear programming problems.

## Linear Programming Method

The linear programming method involves setting up and solving a linear program to find the Nucleolus.

**Time Complexity**: Generally $ O(2^n) $ due to the need to consider all possible coalitions. Modern linear programming solvers can handle this more efficiently.

**Advantages**:
- Provides exact solutions.
- Applicable to a wide range of cooperative games.

**Disadvantages**:
- Computationally intensive.
- Relies on the performance of the linear programming solver.

To compute the Nucleolus, we iteratively solve linear programming problems to find the allocation that minimizes the maximum excess. The excess of a coalition is defined as the difference between the value of the coalition and the sum of the payoffs to its members.


In [4]:
import pulp
import itertools

def linear_programming_nucleolus(players, game_function):
    n = len(players)
    players_list = list(players)
    
    # Initialize the linear programming problem
    lp = pulp.LpProblem("Nucleolus", pulp.LpMinimize)
    
    # Define variables for allocations
    allocations = pulp.LpVariable.dicts("Allocation", players_list, lowBound=0)
    
    # Define the excess variables for each coalition
    excess_vars = {}
    for coalition_size in range(1, n):
        for coalition in itertools.combinations(players_list, coalition_size):
            excess_vars[coalition] = pulp.LpVariable(f"Excess_{coalition}", lowBound=None)
    
    # Define the objective function (minimize the largest excess)
    lp += pulp.lpSum([excess_vars[coalition] for coalition in excess_vars])
    
    # Add constraints for each coalition
    for coalition_size in range(1, n):
        for coalition in itertools.combinations(players_list, coalition_size):
            coalition_set = set(coalition)
            remaining_players = set(players_list) - coalition_set
            for player in remaining_players:
                lp += (
                    pulp.lpSum([allocations[p] for p in coalition_set.union({player})]) 
                    <= game_function(coalition_set.union({player})) + excess_vars[coalition]
                )
                lp += (
                    pulp.lpSum([allocations[p] for p in coalition_set]) 
                    >= game_function(coalition_set) - excess_vars[coalition]
                )
    
    # Solve the linear programming problem
    lp.solve()
    
    # Extract the allocations
    allocations_result = {player: pulp.value(allocations[player]) for player in players_list}
    
    return allocations_result

# Iterative Method

The iterative method approximates the Nucleolus by adjusting allocations to minimize the maximum excess incrementally.

**Time Complexity**: This method generally has a time complexity of $ O(n \cdot 2^n) $.

**Advantages**:
- Simpler to implement than exact methods.
- Provides a good approximation for the Nucleolus.

**Disadvantages**:
- May not converge to the exact Nucleolus.
- Computationally intensive for large games.

The iterative method starts with an initial allocation and repeatedly adjusts the allocations to minimize the maximum excess. This process continues until the allocation cannot be improved further.

In [5]:
import pulp
import itertools

def iterative_nucleolus(players, game_function):
    n = len(players)
    players_list = list(players)
    
    # Initialize the initial allocations
    allocations = {player: 0 for player in players_list}
    
    while True:
        # Initialize the linear programming problem
        lp = pulp.LpProblem("Nucleolus_Iterative", pulp.LpMinimize)
        
        # Define variables for allocations
        allocation_vars = pulp.LpVariable.dicts("Allocation", players_list, lowBound=0)
        
        # Define the excess variables for each coalition
        excess_vars = {}
        for coalition_size in range(1, n):
            for coalition in itertools.combinations(players_list, coalition_size):
                excess_vars[coalition] = pulp.LpVariable(f"Excess_{coalition}", lowBound=None)
        
        # Define the objective function (minimize the largest excess)
        lp += pulp.lpSum([excess_vars[coalition] for coalition in excess_vars])
        
        # Add constraints for each coalition
        for coalition_size in range(1, n):
            for coalition in itertools.combinations(players_list, coalition_size):
                coalition_set = set(coalition)
                lp += (
                    pulp.lpSum([allocation_vars[p] for p in coalition_set]) 
                    >= game_function(coalition_set) - excess_vars[coalition]
                )
                lp += (
                    pulp.lpSum([allocation_vars[p] for p in coalition_set]) 
                    <= game_function(coalition_set) + excess_vars[coalition]
                )
        
        # Solve the linear programming problem
        lp.solve()
        
        # Extract the new allocations
        new_allocations = {player: pulp.value(allocation_vars[player]) for player in players_list}
        
        # Check for convergence
        if all(abs(new_allocations[player] - allocations[player]) < 1e-6 for player in players_list):
            break
        
        allocations = new_allocations
    
    return allocations

# Example and Application

We will apply the Shapley value and Nucleolus algorithms to an example cooperative game with five players in a network communication scenario. The value function for this game is defined as follows:

- Node 1: 2 units
- Node 2: 3 units
- Node 3: 5 units
- Node 4: 7 units
- Node 5: 11 units

The value of a coalition is the sum of the capacities of its members.

## Shapley and Nucleolus Value Calculations

In [6]:
# Example usage:
players = {1, 2, 3, 4, 5}

# Calculate Shapley values for the example game
shapley_values = calculate_shapley_value(players, Game)
print("Shapley Values:", shapley_values)

# Calculate Monte Carlo Shapley values for the example game
shapley_values_mc = monte_carlo_shapley(players, Game, num_simulations=10000)
print("Monte Carlo Shapley Values:", shapley_values_mc)

# Compute the Nucleolus using the Linear Programming Method
nucleolus_lp = linear_programming_nucleolus(players, Game)
print("Nucleolus (Linear Programming):", nucleolus_lp)

# Compute the Nucleolus using the Iterative Method
nucleolus_iterative = iterative_nucleolus(players, Game)
print("Nucleolus (Iterative Method):", nucleolus_iterative)

C:\Users\User\AppData\Local\Temp\ipykernel_860\4281669352.py:8: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  factorial_n = np.math.factorial(n)
C:\Users\User\AppData\Local\Temp\ipykernel_860\4281669352.py:18: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  player_shapley_value += marginal_contribution * np.math.factorial(len(coalition)) * np.math.factorial(n - len(coalition) - 1)


Shapley Values: {1: 2.0, 2: 3.0, 3: 5.0, 4: 7.0, 5: 11.0}
Monte Carlo Shapley Values: {1: 1.9999999999998124, 2: 3.0000000000004805, 3: 4.9999999999999485, 4: 7.000000000000943, 5: 10.999999999998487}
Nucleolus (Linear Programming): {1: 2.0, 2: 3.0, 3: 5.0, 4: 7.0, 5: 11.0}
Nucleolus (Iterative Method): {1: 2.0, 2: 3.0, 3: 5.0, 4: 7.0, 5: 11.0}


# Another Game

## Cost Saving Game Scenario
Consider a scenario where several companies collaborate to share resources and reduce their operational costs. Each company incurs a certain operational cost when working alone. When companies form coalitions, they achieve cost savings due to shared resources. The game function represents the total cost savings achieved by different coalitions of companies.

### Individual Costs
Each company has a defined operational cost when working alone:
- Company 1: $100
- Company 2: $120
- Company 3: $150
- Company 4: $200
- Company 5: $250

### Shared Cost
When companies collaborate, the total cost is reduced by sharing resources. For simplicity, we assume the shared cost is 70% of the sum of individual costs in the coalition.

### Game Function
The value of a coalition is the sum of individual costs minus the shared cost. This represents the cost savings achieved by the coalition.

## Implementation
Below is the implementation of the Cost Saving Game, the calculation of the Shapley values (including the Monte Carlo method), and the two methods for calculating the nucleolus.

In [7]:
def cost_saving_game(coalition):
    individual_costs = {
        1: 100,
        2: 120,
        3: 150,
        4: 200,
        5: 250
    }
    
    # Assume a shared cost when companies collaborate
    shared_cost = 0.7 * sum(individual_costs[company] for company in coalition)
    
    # The value of the coalition is the total individual costs minus the shared cost
    return sum(individual_costs[company] for company in coalition) - shared_cost

# Example usage:
players = {1, 2, 3, 4, 5}

# Compute the Shapley values
shapley_values = calculate_shapley_value(players, cost_saving_game)
print("Shapley Values:", shapley_values)

# Compute the Shapley values using Monte Carlo Method
shapley_values_monte_carlo = monte_carlo_shapley(players, cost_saving_game, num_simulations=10000)
print("Shapley Values (Monte Carlo):", shapley_values_monte_carlo)

# Compute the Nucleolus using the Linear Programming Method
nucleolus_lp = linear_programming_nucleolus(players, cost_saving_game)
print("Nucleolus (Linear Programming):", nucleolus_lp)

# Compute the Nucleolus using the Iterative Method
nucleolus_iterative = iterative_nucleolus(players, cost_saving_game)
print("Nucleolus (Iterative Method):", nucleolus_iterative)

Shapley Values: {1: 29.999999999999996, 2: 36.0, 3: 45.0, 4: 60.0, 5: 75.0}


C:\Users\User\AppData\Local\Temp\ipykernel_860\4281669352.py:8: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  factorial_n = np.math.factorial(n)
C:\Users\User\AppData\Local\Temp\ipykernel_860\4281669352.py:18: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  player_shapley_value += marginal_contribution * np.math.factorial(len(coalition)) * np.math.factorial(n - len(coalition) - 1)


Shapley Values (Monte Carlo): {1: 30.000000000001034, 2: 35.99999999999356, 3: 45.00000000000161, 4: 60.00000000000206, 5: 74.99999999999223}
Nucleolus (Linear Programming): {1: 30.0, 2: 36.0, 3: 45.0, 4: 60.0, 5: 75.0}
Nucleolus (Iterative Method): {1: 30.0, 2: 36.0, 3: 45.0, 4: 60.0, 5: 75.0}


# Conclusion

In this project, we explored the Shapley value and Nucleolus as methods for fair distribution in cooperative games. We provided a historical background, explained the algorithms, and discussed their time complexities, advantages, and disadvantages. Through an example, we demonstrated the computation of these values using both exact and approximation methods.